**Libraries**

In [9]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.oauth2 as oauth2
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns 
%config InlineBackend.figure_format ='retina'
import random
from functools import reduce
from sklearn.ensemble import RandomForestRegressor
from datetime import date
import time

**Spotify API Call**

In [10]:
#used for inserting your username and credentials that you can obtain from spotify developer
client_id= 'ebbb035aedf7428cbdbbd6dd31df89a8'
client_secret= '394032044ced4d8e9df0f9d435f88794'

redirect_uri='http://localhost:8910/callback'
username='224qqnsvbhlau4fhtpl4wpogi'

scope ='user-top-read'


In [ ]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, 
                                                      client_secret=client_secret)
                                                      
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

token = util.prompt_for_user_token(username,scope, client_id, client_secret, redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

**Extract Top Tracks**

In [15]:
results = sp.current_user_top_tracks(limit=50, offset=0,time_range='long_term')

# Convert the results into a dataframe

track_name=[]
track_id=[]
artist=[]
new=[]
album=[]
duration=[]
popularity=[]

for i, items in enumerate(results['items']):
        track_name.append(items['name'])
        track_id.append(items['id'])
        artist.append(items["artists"][0]["name"])
        duration.append(items["duration_ms"])
        album.append(items["album"]["name"])
        popularity.append(items["popularity"])

# Create the final df   
df_top_tracks = pd.DataFrame({ "track_name": track_name, 
                             "album": album, 
                             "track_id": track_id,
                             "artist": artist, 
                             "duration": duration, 
                             "popularity": popularity})

df_top_tracks

,track_name,album,track_id,artist,duration,popularity
0,MIDDLE CHILD,MIDDLE CHILD,2JvzF1RMd7lE3KmFlsyZD8,J. Cole,213593,84
1,Climax (feat. 6lack),On the Run,4N9QsmvQJcGwJHAmDmBttU,Young Thug,189093,65
2,سماع منّي وضل سكار,سماع منّي وضل سكار,4C6gnywwpZWQa8ixhwrSKU,Bassel Sakr,141191,12
3,Stan,East Atlanta Love Letter,1mggAh6xgnKUT6ukw3AU9A,6LACK,303750,59
4,You Are The Reason - Duet Version,Only Human (Special Edition),1x3W8RZxW94lrVGhP95qA6,Calum Scott,190760,73
5,Sin (feat. Jaden Smith),On the Run,1Jl21sidRpEdlVBvYbssLx,Young Thug,202533,65
6,Into You,10X over Riddim (Soca 2019 Trinidad and Tobago...,7pwmtknVJmDfTBSvtzhBEU,Precision Productions,198167,0
7,Whole Wide World,Whole Wide World,6lKKCEeRunxMGM8xgejlyd,Berhana,243160,42
8,Broke,Broke,1JqcZQCnxjDOfZZwQidb1H,Samm Henshaw,207722,65
9,The Box,Please Excuse Me For Being Antisocial,0nbXyq5TXYPCO7pr3N8S4I,Roddy Ricch,196652,92


**Feature Extraction**

In [ ]:
def get_features(sp,df):
    playlist = df[['track_id','track_name']]
    features = []
    
    features += sp.audio_features(playlist.iloc[0:50, 0])

    feature_list = []
    for feature in features:
        feature_list.append([feature['danceability'],
                        feature['energy'],
                        feature['key'],
                        feature['loudness'],
                        feature['mode'],
                        feature['speechiness'],
                        feature['acousticness'],
                        feature['instrumentalness'],
                        feature['liveness'],
                        feature['valence'],
                        feature['tempo'],
                        feature['duration_ms']]
                       )

    df_audio_feature = pd.DataFrame(feature_list, columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 
                                                          'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
                                                          'duration_ms'])
    df_playlist_audio_features = pd.concat([playlist, df_audio_feature], axis=1)
    return df_playlist_audio_features